Copying code from https://google.github.io/mediapipe/solutions/pose#python-solution-api


In [46]:
import os
import cv2
import mediapipe as mp
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

Setting the OSC client:

In [7]:
from pythonosc.udp_client import SimpleUDPClient

ip = "127.0.0.1"
port = 1337

client = SimpleUDPClient(ip, port)  # Create client

Launch the code below:

In [51]:
# For webcam input:
from ast import ListComp


PoseData = list()
cap = cv2.VideoCapture(0)
i = 0
imageName = "Testing"
MaxRecordings = 20
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:

  
  while cap.isOpened():
    
    i += 1
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # prinitng landmarks
    newData = {coord + str(j):lm.__getattribute__(coord)  for j, lm in enumerate(results.pose_landmarks.landmark) for coord in ["x", "y"]}
    PoseData.append(newData)
    # print('first three landmarks', newData)
    # PoseData.append({str(j)+str(key):value  for j, lm in enumerate(results.pose_landmarks.landmark) for key, value in lm.items()})
    # print("X value:", len(results.pose_landmarks.landmark))
    # print("Y value:", results.pose_landmarks.landmark[1].y)
    # print("Z value:", results.pose_landmarks.landmark[1].z)
    # print("Visibility value:", results.pose_landmarks.landmark[1].visibility)

    

    # send value via OSC
    # client.send_message("/pose/landmark/1", [results.pose_landmarks.landmark[1].x, results.pose_landmarks.landmark[1].y, results.pose_landmarks.landmark[1].z])
    # client.send_message("/pose/landmark/2", [results.pose_landmarks.landmark[2].x, results.pose_landmarks.landmark[2].y, results.pose_landmarks.landmark[2].z])
    # client.send_message("test", [rpll[1].z, rpll[1].z, rpll[1].z])
    #client.send_message("/pose/landmark/1", [results.pose_landmarks.landmark[1].x])

    # PoseData.append({four.})
    

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    if (cv2.waitKey(5) & 0xFF == 27) or i>MaxRecordings :
      break
cap.release()
df = pd.DataFrame(PoseData)
df.to_csv(os.path.join("..","data", imageName + ".csv"))

In [36]:
cap.release()